### *Color Detector*
#####  **~DMW Mini Project~**

*Problem Statement:*

There are 16 million colors based on the different RGB color values and we can't remember all of them.
If we want to identify the corol in an image or video then we can't do that on our own as we don't know each of them. 
So we need some tool which can help us to perform the same.
In this project i am trying to find the exact color name which is clicked by mouse. 

In [14]:
import pandas as pd
import cv2

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder


In [8]:
imageUrl = ''
clicked = False
redValue = 0
greenValue = 0
blueValue = 0
xPosition = 0
yPosition = 0

In [9]:
colorNameDataFrame = pd.read_csv('D:\SEM 6th\DWM Lab\HustlerBoy_a_tech_guy_coding_on_computer_anime_character_style__d694b893-17ed-4f3c-aaf5-9f7b54245afd.png')
colorNameDataFrame.drop(colorNameDataFrame.iloc[:,5:8], inplace=True, axis=1)
colorNameDataFrame.rename(columns={'Hex (24 bit)':'Hex', 'Red (8 bit)':'Red', 'Green (8 bit)':'Green', 'Blue (8 bit)':'Blue'}, inplace=True)
image = cv2.imread(imageUrl)




# Prepare training data for regression
X_train = colorNameDataFrame[['Red', 'Green', 'Blue']]
y_train = colorNameDataFrame['Name']

# Encode color names to numeric labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Train the regression model
regression_model = LinearRegression()
regression_model.fit(X_train, y_train_encoded)

In [10]:
def getColorName(red, green, blue):
    rgb_values = np.array([[red, green, blue]])
    color_label = regression_model.predict(rgb_values)
    color_name = label_encoder.inverse_transform(color_label)
    return color_name[0]



def getColorName(red,green,blue):
    minimumValue = 10000
    for i in range(len(colorNameDataFrame)):
        rgbValue = abs(red- int(colorNameDataFrame.loc[i,"Red"])) + abs(green- int(colorNameDataFrame.loc[i,"Green"]))+ abs(blue- int(colorNameDataFrame.loc[i,"Blue"]))
        if(rgbValue <= minimumValue):
            minimumValue = rgbValue
            colorName = colorNameDataFrame.loc[i,"Name"]
    return colorName


In [12]:
def draw_function(event, x,y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global blueValue, greenValue, redValue, xPosition, yPosition, clicked
        clicked = True
        xPosition = x
        yPosition = y
        blueValue, greenValue, redValue = image[yPosition, xPosition]
        blueValue = int(blueValue)
        greenValue = int(greenValue)
        redValue = int(redValue)

In [16]:
if __name__ == '__main__':
    cv2.namedWindow('Color Name')
    cv2.setMouseCallback('Color Name', draw_function)
    
    while (1):        
        if (clicked):            
            cv2.rectangle(image, (20, 20), (950, 60), (blueValue, greenValue, redValue), -1)
            colorName = 'Selected color name is:-' + getColorName(redValue, greenValue, blueValue)
            cv2.putText(image, colorName, (50, 50), 2, 0.75, (255, 255, 255), 1, cv2.FONT_ITALIC)
            minimumValue = abs(redValue + greenValue + blueValue)
            if (minimumValue >= 600):
                cv2.putText(image, colorName, (50, 50), 2, 0.75, (0, 0, 0), 1, cv2.FONT_ITALIC)
            clicked = False
        cv2.imshow("Color Name", image)
        # Break the loop when user hits 'esc' key
        if cv2.waitKey(20) & 0xFF == 27:
            break
    cv2.destroyAllWindows()